# Model Training for PianoBear Transcriber

In [21]:
import numpy as np
import os
from sklearn.model_selection import train_test_split

def find_max_length(data_dir):
    max_len = 0
    file_paths = [os.path.join(data_dir, f) for f in os.listdir(data_dir) if f.startswith('processed_maestro_data')]
    for path in file_paths:
        data = np.load(path, allow_pickle=True)
        for item in data:
            spectrogram = item['audio_spectrogram']
            if spectrogram.shape[1] > max_len:  # 스펙트로그램 길이는 2차원 배열의 두 번째 차원의 크기
                max_len = spectrogram.shape[1]
    return max_len

data_dir = 'maestro_data'
max_length = find_max_length(data_dir)
print(f"The maximum length of the spectrograms is: {max_length}")

def pad_sequences(sequences, maxlen, dtype='float32', padding='post', truncating='post', value=0.0):
    num_samples = len(sequences)
    num_frequency_bins = sequences[0].shape[0]  # 주파수 빈도수, 여기서는 1025
    x = np.full((num_samples, num_frequency_bins, maxlen), value, dtype=dtype)
    for idx, s in enumerate(sequences):
        if truncating == 'pre':
            trunc = s[:, -maxlen:]
        else:
            trunc = s[:, :maxlen]
        if padding == 'post':
            x[idx, :, :trunc.shape[1]] = trunc
        else:
            x[idx, :, -trunc.shape[1]:] = trunc
    return x

def pad_labels(labels, max_len, num_keys=88, value=0):
    num_samples = len(labels)
    padded_labels = np.full((num_samples, max_len, num_keys), value)
    for i, label_seq in enumerate(labels):
        # label_seq가 리스트일 경우에 NumPy 배열로 변환
        if isinstance(label_seq, list):
            label_array = np.array(label_seq)
        else:
            label_array = label_seq
        
        # label_array가 1차원일 경우, 2차원으로 변환
        if label_array.ndim == 1:
            label_array = label_array.reshape(-1, num_keys)

        length = min(max_len, label_array.shape[0])
        padded_labels[i, :length, :] = label_array[:length, :]
    return padded_labels

def encode_midi(midi_notes):
    """
    midi_notes 리스트를 입력받아 피아노의 88개 키에 대한 활성화 상태를 나타내는 이진 벡터로 변환합니다.
    """
    num_keys = 88  # 88개 키
    encoded = np.zeros(num_keys, dtype=np.int32)  # 88개 키의 초기 상태
    for note_info in midi_notes:
        index = note_info['note'] - 21  # MIDI 노트 21(A0)부터 인덱싱
        if 0 <= index < num_keys:
            if note_info['velocity'] > 0:
                encoded[index] = 1  # note_on 이벤트, 벨로시티가 0보다 크면 활성화
            else:
                encoded[index] = 0  # 벨로시티가 0이면 비활성화

    return encoded

def data_generator(file_paths, batch_size=32, max_len=None):
    while True:
        for path in file_paths:
            data = np.load(path, allow_pickle=True)
            batch_features = []
            batch_labels = []
            for i in range(0, len(data), batch_size):
                batch_data = data[i:i+batch_size]
                features = [x['audio_spectrogram'] for x in batch_data]
                labels = [encode_midi(x['midi_notes']) for x in batch_data]

                padded_features = pad_sequences(features, max_len)
                padded_labels = np.array(labels)  # 레이블은 이미 모든 타임 스텝에 대해 인코딩 되었기 때문에 추가 패딩은 필요 없음

                if len(batch_features) >= batch_size:
                    yield (np.array(padded_features).reshape(batch_size, max_len, -1),
                           np.array(padded_labels).reshape(batch_size, num_keys))  # 레이블 차원 수정
                    batch_features = []
                    batch_labels = []



# 파일 경로 리스트 생성
data_dir = 'maestro_data'
file_paths = [os.path.join(data_dir, f) for f in os.listdir(data_dir) if f.startswith('processed_maestro_data')]

# 데이터 파일들을 훈련, 검증, 테스트 세트로 분할
train_files, test_files = train_test_split(file_paths, test_size=0.3, random_state=42)
val_files, test_files = train_test_split(test_files, test_size=0.33, random_state=42)

# 제네레이터 생성
train_gen = data_generator(train_files, max_len=max_length)
val_gen = data_generator(val_files, max_len=max_length)
test_gen = data_generator(test_files, max_len=max_length)


In [20]:
import os
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, TimeDistributed, Bidirectional

os.environ["CUDA_VISIBLE_DEVICES"] = "0" # GPU
# os.environ["CUDA_VISIBLE_DEVICES"] = "-1" # CPU

def build_model(input_shape, output_dim):
    model = Sequential([
        Bidirectional(LSTM(64, return_sequences=True, input_shape=input_shape)),
        Dropout(0.5),
        TimeDistributed(Dense(output_dim, activation='sigmoid'))
    ])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

# 모델 구성
input_shape = (None, 2048 / 2 + 1)  # 예: (시간 단계 수, 특성 수)
output_dim = 88  # 88개 피아노 키
model = build_model(input_shape, output_dim)

# 모델 학습
model.fit(train_gen, steps_per_epoch=50, epochs=20, validation_data=val_gen, validation_steps=10)


StopIteration: 

In [10]:
# 데이터 로딩 및 구조 확인
sample_data = np.load('maestro_data/processed_maestro_data_0_49.npy', allow_pickle=True)
print(type(sample_data[0]))  # 첫 번째 아이템의 타입 출력
print(sample_data[0].keys())  # 첫 번째 아이템의 키 출력
print(type(sample_data[0]['midi_notes']))  # midi_notes 필드의 타입 출력
print(sample_data[0]['midi_notes'])  # midi_notes 내의 키 확인 (만약 딕셔너리라면)

<class 'dict'>
dict_keys(['midi_notes', 'audio_spectrogram'])
<class 'list'>
[{'note': 67, 'velocity': 52, 'time': 755, 'type': 'note_on'}, {'note': 72, 'velocity': 67, 'time': 615, 'type': 'note_on'}, {'note': 67, 'velocity': 0, 'time': 20, 'type': 'note_on'}, {'note': 72, 'velocity': 0, 'time': 74, 'type': 'note_on'}, {'note': 78, 'velocity': 65, 'time': 11, 'type': 'note_on'}, {'note': 71, 'velocity': 45, 'time': 2, 'type': 'note_on'}, {'note': 61, 'velocity': 39, 'time': 5, 'type': 'note_on'}, {'note': 67, 'velocity': 39, 'time': 2, 'type': 'note_on'}, {'note': 67, 'velocity': 0, 'time': 206, 'type': 'note_on'}, {'note': 61, 'velocity': 0, 'time': 64, 'type': 'note_on'}, {'note': 78, 'velocity': 0, 'time': 274, 'type': 'note_on'}, {'note': 79, 'velocity': 58, 'time': 383, 'type': 'note_on'}, {'note': 71, 'velocity': 0, 'time': 173, 'type': 'note_on'}, {'note': 79, 'velocity': 0, 'time': 73, 'type': 'note_on'}, {'note': 79, 'velocity': 69, 'time': 250, 'type': 'note_on'}, {'note': 6